In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv
import os
import datetime
import time
import random
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
pd.options.display.float_format = '{:.0f}'.format
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\c_the\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [12]:
def cap_finder(cap_string):
    cap = float(cap_string[0:-1].replace(',',''))
    if cap_string[-1] == 'B':
        cap = cap*10**9
    elif cap_string[-1] == 'M':
        cap = cap*10**6
    elif cap_string[-1] == 'T':
        cap = cap*10**12
    return cap

def close_date_func(close_day):
    today = datetime.date.today()
    while close_day != today.day:
        today = today - datetime.timedelta(days = 1)
    return today

In [13]:
def open_link(symbol):
    is_link = 'https://ca.finance.yahoo.com/quote/'+ symbol +'.TO/key-statistics?p=' + symbol + '.TO&.tsrc=fin-srch'
    
    #driver = webdriver.Chrome()
    driver.get(is_link)
    time.sleep(random.uniform(0.5, 3))
    html = driver.execute_script('return document.body.innerHTML;')
    soup = BeautifulSoup(html,'lxml')
    return soup

In [14]:
def collect_data(symbol, soup):
    close_price = [entry.text for entry in soup.find_all('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})]
    close_price = float(close_price[0].replace(',',''))
    close_price
    
    metrics = [entry.text for entry in soup.find_all('td', {'class':'Fw(500) Ta(end) Pstart(10px) Miw(60px)'})]
    if metrics[0] == 'N/A':
        cap = 'N/A'
    else:
        cap = cap_finder(metrics[0])
    if metrics[2] == 'N/A':
        P_to_E = metrics[2]
    else:
        P_to_E = float(metrics[2].replace(',',''))
    if metrics[4] == 'N/A':
        PEG = metrics[4]
    else:
        PEG = float(metrics[4].replace(',',''))
    if metrics[6] == 'N/A':
        P_to_B = metrics[6]
    else:
        P_to_B = float(metrics[6].replace(',',''))
    if metrics[28] == 'N/A':
        Dividend = metrics[28]
    else:
        Dividend = cap_finder(metrics[28])/100
    if metrics[32] == 'N/A':
        Payout_ratio = metrics[32]
    else:
        Payout_ratio = cap_finder(metrics[32])/100
    
    
    full_name = [entry.text for entry in soup.find_all('h1', {'class':'D(ib) Fz(18px)'})]
    full_name = full_name[0]
    name = full_name.split('(')[0].strip()
    name = name.replace(',','')
    
    #symbol = full_name.split('(')[1]
    #symbol = symbol.split('.')[0]
    
    if P_to_E == 'N/A' or P_to_B == 'N/A':
        value = 'N/A'
    elif Dividend == 'N/A':
        value = P_to_E*P_to_B
    else:
        value = P_to_E*P_to_B/(1+Dividend)
    
    
    date_info = [entry.text for entry in soup.find_all('span', {'data-reactid':'35'})]
    date_info = date_info[0].split(':')[1].strip()
    date_info = date_info.split(' ')
    try:
        close_day = int(date_info[1])
        close_date = close_date_func(close_day)
    except:
        close_date = datetime.date.today()
    
    return [name, symbol, close_date.year, close_date.month, close_date.day, close_price,\
            cap, P_to_E, PEG, P_to_B, Dividend, Payout_ratio, value]

In [15]:
def store_data(symbol, important_metrics):
    if os.path.isfile('Data/' + symbol + '-TO.csv'):
        with open('Data/' + symbol +'-TO.csv', mode = 'r', newline = '') as curr_file:
            csv_reader = csv.reader(curr_file, delimiter = ',')
            for line in csv_reader:
                if line[7] == 'Industry':
                    continue
                else:
                    sector = line[6]
                    industry = line[7]
                    break
            curr_file.close()
            
        with open('Data/' + symbol +'-TO.csv', mode = 'a', newline = '') as curr_file:
            curr_writer = csv.writer(curr_file, delimiter = ',')
            curr_writer.writerow(important_metrics[0:6] + [sector, industry] + important_metrics[6:])
            curr_file.close()
            
    else:
        is_link = 'https://ca.finance.yahoo.com/quote/' + symbol + '.TO/profile?p=' + symbol + '.TO'
        driver.get(is_link)
        html = driver.execute_script('return document.body.innerHTML;')
        soup = BeautifulSoup(html,'lxml') 
        
        industrys = [entry.text for entry in soup.find_all('span', {'class':'Fw(600)'})]
        try:
            industry = industrys[-2]
            if len(industry) == 0:
                industry = 'N/A'
        except:
            industry = 'N/A'
        try:
            sector = industrys[0]
            if len(sector) == 0:
                sector = 'N/A'
        except:
            sector = 'N/A'
        
        with open('Data/' + symbol + '-TO.csv', mode = 'w', newline = '') as curr_file:
            curr_writer = csv.writer(curr_file, delimiter = ',')
            curr_writer.writerow(['Company Name', 'Symbol', 'Year', 'Month', 'Day', 'Close Price', 'Sector',\
                                  'Industry', 'Market Cap', 'Price to Earnings', 'PEG',\
                                  'Price to Book', 'Dividend', 'Payout Ratio', 'Value Metric'])
            curr_writer.writerow(important_metrics[0:6] + [sector, industry] + important_metrics[6:])
            curr_file.close()
    
    return important_metrics[0:6] + [sector, industry] + important_metrics[6:]

In [16]:
def open_list():
    symbol_list = []
    with open('symbol list.csv', mode = 'r', newline = '') as f:
        csv_reader = csv.reader(f, delimiter = ',')
        for line in f:
            symbol_list.append(line.strip())
        f.close()

    return symbol_list

In [17]:
def mine_market():
    start_time = datetime.datetime.now()
    symbol_list = open_list()
    todays_list = []
    symbol_list = [x for x in symbol_list if x not in todays_list]
    while len(symbol_list) > 0:
        try:
            random.shuffle(symbol_list)
            counter = random.randint(0,30)
            for symbol in symbol_list:
                counter += 1
                if counter == 100:
                    counter = random.randint(0,30)
                    time.sleep(3*counter)
                print(symbol + ', ', end = '')
                soup = open_link(symbol)
                try: 
                    important_metrics = collect_data(symbol, soup)
                except:
                    time.sleep(30)
                    soup = open_link(symbol)
                    importand_metrics = collect_data(symbol, soup)
                important_metrics = store_data(symbol, important_metrics)
                todays_list.append(symbol)
        finally:
            symbol_list = [x for x in symbol_list if x not in todays_list]
    print()
    print('Data Stored!!')
    print('Runtime: ' + str(datetime.datetime.now() - start_time))

In [18]:
def mining_correction(symbol_list):
    start_time = datetime.datetime.now()
    todays_list = []
    symbol_list = [x for x in symbol_list if x not in todays_list]
    
    for symbol in symbol_list:
        f = open('Data/' + symbol +'-TO.csv', "r+")
        lines = f.readlines()
        lines.pop()
        f = open('Data/' + symbol +'-TO.csv', "w+")
        f.writelines(lines)

    while len(symbol_list) > 0:
        try:
            random.shuffle(symbol_list)
            counter = random.randint(0,30)
            for symbol in symbol_list:
                counter += 1
                if counter == 100:
                    counter = random.randint(0,30)
                    time.sleep(3*counter)
                print("'" + symbol + "'" + ', ', end = '')
                soup = open_link(symbol)
                try: 
                    important_metrics = collect_data(symbol, soup)
                except:
                    time.sleep(30)
                    soup = open_link(symbol)
                    importand_metrics = collect_data(symbol, soup)
                important_metrics = store_data(symbol, important_metrics)
                todays_list.append(symbol)
        finally:
            symbol_list = [x for x in symbol_list if x not in todays_list]
    print()
    print('Data Updated!!')
    print('Runtime: ' + str(datetime.datetime.now() - start_time))

In [19]:
#time.sleep(5000)
mine_market()

ALYA, TV, APY, DIV, ORL, NCU-WT, FTT, SMT, CIX, BOS, IPCI, PTI-UN, AEZS, RY, IPO, PMB-UN, AGI, PEY, GDL, HLF, TECK-A, PBY-UN, SZLS-WS, TLO, VII, RBN-UN, MRD, X, LBS, VCM, FRU-R, CRH, LGD, TGOD-WR, JAG, ARG, WRX, BGI-UN, OSK, SBN, COG, IPCO, JOSE, WCM-B, GXE, JOY, DII-B, LFE, UEX, MBA, SPPP, MND, INO-UN, ENGH, WCM-A, DF, ELF, DS, EFL, S, ADW-B, K, NVCN, AX-UN, TA, TBL, TH, PRMW, PRV-UN, DR, AW-UN, ISV, BK, H, BNE, YRI, ARE, SRHI, ONEX, ORV, IMV, VNP, CHE-UN, FCR-UN, TNT-UN, ESN, GOLD, MRU, PSK, TZS, CPI, NXJ, TXG, WIR-UN, CDAY, CNE, ACQ, GBT, GLXY, IIP-UN, MAV, BHC, PSD, PFB, TV-WT, JE, DIAM, HCG, NVA, CXI, NDM, ATD-B, SOY, MPVD, CHP-UN, CGI, SBI, L, ATA, SGR-UN, NPI, CAS, NEO, EDT, FFI-UN, DXT, DRM, BDT, AVCN, CWL, QBTC-U, GSY, CTC, PD, GATO, PBL, OPS, CIQ-UN, GWR, IDG, QSR, REAL, BCE, MI-UN, SES, PBH, MKP, POM, TCL-A, TOT, HSM, NTR, III, SIA, WPRT, TRL, IMG, WRN, TKO, IN, INE, AVP, FFN, ERO, MPCT-UN, QTRH, SKE, RNW, FVI, FCU, KL, IAG, CRDL, WLLW, GDV, HSE, FOOD, BC-U, CVG, USF-UN, GEN

In [21]:
symbol_list = open_list()
tsx_df = None
min_len = 16
max_len = 0
off_syms = []
for symbol in symbol_list:
    with open('Data/' + symbol +'-TO.csv', mode = 'r', newline = '') as curr_file:
        curr_data = curr_file.readlines()[-1]
        curr_file.close()
    curr_data = curr_data.strip().split(',')
    min_len = min(len(curr_data), min_len)
    max_len = max(len(curr_data), max_len)
    if len(curr_data) != 15:
        off_syms.append(symbol)
    if tsx_df is not None:
        tsx_df.loc[len(tsx_df)] = curr_data
        #tsx_df = tsx_df.append([curr_data])
    else:
        tsx_df = pd.DataFrame([curr_data], columns = ['Company Name', 'Symbol', 'Year',\
                                                        'Month', 'Day', 'Closing Price', 'Sector', 'Industry',\
                                                        'Market Cap', 'Price to Earnings', 'PEG',\
                                                        'Price to Book', 'Dividend', 'Payout Ratio', 'Value Metric'])
tsx_df.to_csv('tsx.csv', encoding = 'utf-8', index = False)
print('Data ready for analysis!!')
print(min_len)
print(max_len)
print(off_syms)

Data ready for analysis!!
15
15
[]


In [41]:
symbol_list = ['DIR-UN', 'VMD', 'BGI-UN', 'LN']
mining_correction(symbol_list)

VMD, DIR-UN, BGI-UN, LN, 

UnboundLocalError: local variable 'sector' referenced before assignment

In [37]:
tsx_df.sort_values(by =['Value Metric'], ascending = True)
tsx_df.head(20)

,Company Name,Symbol,Year,Month,Day,Closing Price,Sector,Industry,Market Cap,Price to Earnings,PEG,Price to Book,Dividend,Payout Ratio,Value Metric
0,Lucara Diamond Corp.,LUC,2021,1,7,0.63,Basic Materials,Other Precious Metals & Mining,250050000.0,N/A,10.46,1.27,N/A,0.0,N/A
1,Enterprise Group Inc.,E,2021,1,7,0.22,Industrials,Engineering & Construction,10950000.0,N/A,N/A,0.29,N/A,0.0,N/A
2,Urbana Corporation,URB,2021,1,6,2.93,Financial Services,Asset Management,162280000.0,56.35,N/A,0.59,0.030699999999999998,1.6,32.25623362763171
3,Buhler Industries Inc.,BUI,2021,1,6,2.7,Industrials,Farm & Heavy Construction Machinery,67500000.0,N/A,N/A,0.83,N/A,0.0,N/A
4,Gold Standard Ventures Corp,GSV,2021,1,7,0.93,Basic Materials,Gold,293100000.0,N/A,-9.1,1.13,N/A,0.0,N/A
5,Endeavour Silver Corp.,EDR,2021,1,7,6.82,Basic Materials,Silver,1070000000.0000001,N/A,-0.14,7.82,N/A,0.0,N/A
6,Opsens Inc.,OPS,2021,1,7,1.39,Healthcare,Medical Instruments & Supplies,125490000.0,N/A,N/A,8.18,N/A,0.0,N/A
7,M Split Corp.,XMF-A,2021,1,6,0.49,Financial Services,Asset Management,1120000.0,N/A,N/A,N/A,N/A,0.0,N/A
8,Chesswood Group Limited,CHW,2021,1,7,8.91,Financial Services,Credit Services,145100000.0,N/A,N/A,1.18,0.0472,N/A,N/A
9,Life & Banc Split Corp.,LBS,2021,1,7,7.3,Financial Services,Asset Management,199260000.0,N/A,N/A,1.91,0.16670000000000001,N/A,N/A


In [197]:
# This block can be used to revert to yesterday

#symbol_list = open_list()
#for symbol in symbol_list:
#    f = open('Data/' + symbol +'-TO.csv', "r+")
#    lines = f.readlines()
#    lines.pop()
#    f = open('Data/' + symbol +'-TO.csv', "w+")
#    f.writelines(lines)